# Read image stack

In [ ]:
import imageio
from pathlib import Path

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

from tqdm.auto import tqdm

from astro3d import image_stack, get_output

## Settings

We again need to set the printer-specific settings and the directory of the slides:

In [ ]:
dpi_x = 600
dpi_y = 300
dz    = 27e-4 # 27 micron
# directory = 'slices_turbulent_box'
directory = Path(get_output()) / 'slices_mhd'

## Read Images

Get number of files and file dimensions

In [ ]:
files = sorted(list(directory.glob('*.png')))
im = imageio.imread(files[len(files)//2])
_x = np.arange(im.shape[1])
_y = np.arange(im.shape[0])

dx = 2.54 / dpi_x
dy = 2.54 / dpi_y
dpi_z = 2.54 / dz

## Print out statistics

Sum up the individual colors other than white. This can take a few minutes for large stacks.

In [ ]:
cols = image_stack.check_colors(im)
%time summed = image_stack.image_sum(files, colors=cols)#[:-1])

Check that every pixel is assigned one color and not more

In [ ]:
print(f'Material assignment is {int(summed.sum() != len(files) * np.prod(im.shape[:2])) * "NOT "}OK')

Show the distribution of materials

In [ ]:
nc = len(cols) - 1

f, ax = plt.subplots(2, nc, figsize=(5 * nc, 5), gridspec_kw={'height_ratios': [1, 20]}, dpi=150)

for ic, col in enumerate(cols[:-1]):
    cmap = LinearSegmentedColormap.from_list('my', [[1, 1, 1], cols[ic] / 255])
    cc = ax[1, ic].imshow(summed[:, :, ic], vmin=0, vmax=1, origin='lower', cmap=cmap)
    ax[1, ic].set_aspect(0.5)
    f.colorbar(cc, cax=ax[0, ic], orientation='horizontal');

In [ ]:
print(f'{len(files)} files')
print(f'dimension = {len(_x) * dx:.2f} x {len(_y) * dy:.2f} x {len(files) * dz:.2f} cm')
print(f'nr of empty pixels: {(summed[:,:,:-1].sum(-1)==0).sum() / np.prod(summed.shape[:2]):.2%}')
print(f'filling fraction: {(summed[:,:,:-1].sum(-1)).sum() / (len(files) * np.prod(summed.shape[:2])):.2%}')
print(f'most opaque pixel has {summed[:,:,:-1].sum(-1).max():n} filled pixels (={summed[:, :, :-1].sum(-1).max() / len(files):.2%} of all layers are filled)')

plt.imshow([cols]).axes.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
print('mean counts in non-transparent columns: ' + ', '.join([f'{np.mean(summed[:,:,i][summed[:,:,i]!=0]):.2g}' for i in range(len(cols))]))

In [ ]:
f, ax = plt.subplots()
_= ax.hist(summed[:,:, 0].ravel(), bins=np.linspace(0,20,21)-0.5);
ax.set_yscale('log')
ax.set_xlim(right=20);

## Plots

### Statistics plots
Show fully transparent pixels and the density distribution.

In [ ]:
f, axs = plt.subplots(1, 3, figsize=(11, 3), dpi=100, tight_layout=True)

opts = dict(cmap='gray', origin='upper')
summed_image = summed[:, :, :-1].sum(-1)

ax = axs[0]
cc=ax.imshow(summed_image.T, **opts, vmin=0, vmax=7)
ax.set_aspect(dpi_x / dpi_y)
plt.colorbar(cc, ax=ax).set_label('# of opaque pixes along LOS')

ax = axs[1]
i = ax.imshow(summed_image.T, vmin=0, vmax=summed_image.max(), **opts)
ax.set_aspect(dpi_x / dpi_y)
plt.colorbar(i, ax=ax).set_label('# of opaque pixes along LOS')

ax = axs[-1]
counts, bins, patches = ax.hist(summed_image.ravel(), bins=np.arange(summed_image.max()))
ax.set_yscale('log')
ax.set_xlabel('number of filled voxels in column')
ax.set_xlabel('count');

# Top color

Compute the first color along each column as estimate of what the plot might look like

In [ ]:
im = imageio.imread(files[0])

dummyval = 300

im_set = dummyval * np.ones(im.shape, dtype=np.int32)
bg = np.array([255] * 3, dtype=np.int32)
empty = np.array([dummyval] * 3, dtype=np.int32)

for file in tqdm(files):
    im = imageio.imread(file)
    im_set = np.where((np.all(im_set==empty[None, None, :], -1) &  ~np.all(im==bg[None, None, :], -1))[:,:,None], im, im_set)

In [ ]:
f, ax = plt.subplots(dpi=300)
ax.imshow(np.minimum(np.maximum(im_set, 0), 255))
ax.set_aspect(dpi_x / dpi_y);